# Data Science in Phyton

## Table of Content
- [Лекция 1: Объектно-ориентированное программирование](#Лекция1)
- [Лекция 2: Функциональное программирование](#Лекция2)
- [Лекция 3: Параллельное программирование](#Лекция3)
- [Лекция 4: Скорость вычислений](#Лекция4)
    - [Задачка про параллельные вычисления](#Задачка1)
- [Лекция 5: Многопользовательская ОС](#Лекция5)

 # Лекция 1: Объектно-ориентированное программирование <a name="Лекция1"></a>

[лекция здесь](https://www.youtube.com/watch?v=Dx2SE4hYy4g&t=3s&ab_channel=%D0%A2%D0%B8%D0%BC%D0%BE%D1%84%D0%B5%D0%B9%D0%A5%D0%B8%D1%80%D1%8C%D1%8F%D0%BD%D0%BE%D0%B2)

 # Лекция 2: Функциональное программирование <a name="Лекция2"></a>

[лекция здесь](https://www.youtube.com/watch?v=3Dmi4b8MkMM&t=1s&ab_channel=%D0%A2%D0%B8%D0%BC%D0%BE%D1%84%D0%B5%D0%B9%D0%A5%D0%B8%D1%80%D1%8C%D1%8F%D0%BD%D0%BE%D0%B2)

 # Лекция 3: Параллельное программирование <a name="Лекция3"></a>

[лекция здесь](https://www.youtube.com/watch?v=QitEF7Qvi4w&ab_channel=%D0%A2%D0%B8%D0%BC%D0%BE%D1%84%D0%B5%D0%B9%D0%A5%D0%B8%D1%80%D1%8C%D1%8F%D0%BD%D0%BE%D0%B2)

 # Лекция 4: Скорость вычислений <a name="Лекция4"></a>

[ссылка на лекцию](https://www.youtube.com/watch?v=UA4Wsp3AWYc&ab_channel=%D0%A2%D0%B8%D0%BC%D0%BE%D1%84%D0%B5%D0%B9%D0%A5%D0%B8%D1%80%D1%8C%D1%8F%D0%BD%D0%BE%D0%B2)

## Задачка про параллельные вычисления <a name="Задачка1"></a>

см здесь: https://youtu.be/QitEF7Qvi4w?t=6925

### Дано: 
массив прямоугольной формы а х в, заполненный нулями кроме верхней строки и левого столбца - они заполнены целыми числами от 1 до а и в соответственно. 

### Задание:
1. Заменить в ячейках массива нули на числа, равные сумме содержимого верхней и левой ячеек.
2. Сделать то же самое в параллельном вычислении.

In [130]:
import numpy as np
import pandas as pd
import time            # модуль для органзации задержки по времени sleep
# import timeit          модуль для вычисления времени выполнения программы

[timeit](https://docs.python.org/3/library/timeit.html) doc  
[timeit](https://dev-gang.ru/article/kak-izmerit-vremja-vypolnenija-skripta-python-ub9t58z05h/) example  
timeit на больших кусках кода отказался работать. Буду использовать time.

In [135]:
# готовим массив
a = 9  # выбираем число строк массива
b = 5  # выбираем число столбцов массива
M = np.zeros(shape=(a,b), dtype=np.int32) # создаем массив размером a x b заполняем его нулями

for i in range(a):    # первый столбец заполняем целыми числами от 1 и далее 
    M[i,0]=i+1
    
for j in range(b):    # первую строку заполняем целыми числами от 1 и далее 
    M[0,j]=j+1
    
M

array([[1, 2, 3, 4, 5],
       [2, 0, 0, 0, 0],
       [3, 0, 0, 0, 0],
       [4, 0, 0, 0, 0],
       [5, 0, 0, 0, 0],
       [6, 0, 0, 0, 0],
       [7, 0, 0, 0, 0],
       [8, 0, 0, 0, 0],
       [9, 0, 0, 0, 0]], dtype=int32)

### Решение без параллельных вычислений

In [137]:
R = np.copy(M)                    # создаем копию массива чтобы упростить отладку

def cell(i, j, R):
    """
    вычисляет значение следующей ячейки массива R по значениям двух предыдущих ячеек (слева и сверху)
    """
    time.sleep(0.25)              # задержка имитирует очень сложное вычисление в ячейке
    return R[i-1,j] + R[i,j-1]

start_time = time.time()          # фиксируем время начала работы цикла

for i in range(1, a):  # не используем zip чтобы впоследствии можно было выбирать направление проходки
    for j in range(1, b):
        R[i,j] = cell(i, j, R)
        print("Next cell calculation completed: i={}, j={}, R={}".format(
            i, j, R[i, j]), " "*100, end="\r")

all_time = time.time() - start_time           # фиксируем время конца цикла, посчитали длительность
print("\n", "Time elapsed = ", all_time, "seconds")  # выводим длительность
        
R

Next cell calculation completed: i=8, j=4, R=1507                                                                                                     
 Time elapsed =  8.085600852966309 seconds


array([[   1,    2,    3,    4,    5],
       [   2,    4,    7,   11,   16],
       [   3,    7,   14,   25,   41],
       [   4,   11,   25,   50,   91],
       [   5,   16,   41,   91,  182],
       [   6,   22,   63,  154,  336],
       [   7,   29,   92,  246,  582],
       [   8,   37,  129,  375,  957],
       [   9,   46,  175,  550, 1507]], dtype=int32)

### Решение с параллельными вычислениями

Идея: пока вычисляется i-й ряд (1<i<a) можно организовать параллельное вычисление i+1 ряда с задержкой на 1 такт.  
Лекцию см здесь: https://youtu.be/QitEF7Qvi4w?t=5326  
модуль [\_thread](https://docs.python.org/3/library/_thread.html?highlight=thread#module-_thread)  
модуль [threading](https://docs.python.org/3/library/threading.html?highlight=thread#module-threading)

 # Лекция 5: Многопользовательская ОС <a name="Лекция5"></a>

[ссылка на лекцию](https://youtu.be/QC73lKmJS0s)

Как запускаются вычислительные процессы в операционной системе  
- пользователи и группы пользователей
- асинхронные вычисления
- все ресурсы в Linux - это файлы
- через взаимодействие с оболочкой через "born again shell" (bash) или "z-shell" (zsh)  

Пример запустим процесс и выведем результат не на экран, а в файл - НО средствами ОС: [тыц](https://youtu.be/QC73lKmJS0s?t=3852)
    